In [4]:
import numpy as np 
from matplotlib import pyplot as plt 
import os 
from pathlib import Path 


In [20]:
base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments/MNIST_STRATIFIED_CLASSIFIERS_MADGAN/'
experiments = os.listdir(strat_exp_path)

experiments[0]

'2025-02-25_Stratified_classifierExperiment_MNIST__10_used_generator_0__images_real_3000_gen_2000'

In [38]:
def extract_info_from_experiment_name(exp: str) -> {}: 
    splt = exp.split('_')
    ret = {'dataset': splt[3], 'n_gen': splt[5], 'used_gen': splt[8], 'n_real': splt[-3], 'n_fake': splt[-1]}

    return ret

In [48]:
# load all histories: 

histories = {}
meta_info = {}

for exp in experiments: 
    meta = extract_info_from_experiment_name(exp)
    if meta['n_gen'] == str(10): 
        histories[exp] = np.load(Path(strat_exp_path) / experiments[0] / 'training_history.npy', allow_pickle=True).item()
        meta_info[exp] = meta
    


In [49]:
len(meta_info)

50